In [19]:
import pandas as pd
from tqdm import tqdm
import networkx as nx
from networkx import from_pandas_dataframe

ImportError: cannot import name 'from_pandas_dataframe' from 'networkx' (/home/simone/anaconda3/lib/python3.9/site-packages/networkx/__init__.py)

In [5]:
a_q=pd.read_csv('a2q',sep=' ', names=['u','v','t'])  #answers to questions
c_a=pd.read_csv('c2a',sep=' ', names=['u','v','t'])  #comments to answers
c_q=pd.read_csv('c2q',sep=' ', names=['u','v','t'])  #comments to questions

In [6]:
print(a_q.shape,c_a.shape,c_q.shape)

(17823525, 3) (25405374, 3) (20268151, 3)


In [7]:
a_q=a_q.sort_values('t')
c_a=c_a.sort_values('t')
c_q=c_q.sort_values('t')

#change time to hours
a_q['t']=a_q['t'].div(3600)
c_a['t']=c_a['t'].div(3600)
c_q['t']=c_q['t'].div(3600)

In [8]:
print(a_q.isnull().sum())
print(c_a.isnull().sum())
print(c_q.isnull().sum())
#No missing values

u    0
v    0
t    0
dtype: int64
u    0
v    0
t    0
dtype: int64
u    0
v    0
t    0
dtype: int64


In [12]:
a_q['type']='a_q'
c_a['type']='c_a'
c_q['type']='c_q'

In [16]:
def time_maxmin(data,min,max): #Retrieve maximum and minimum timestamps
    m=data['t'].min()
    M=data['t'].max()
    if m<min:
        min=m
    if M>max:
        max=M
    return min,max

In [17]:
m=10**(10)
M=0
m, M = time_maxmin(a_q, m, M)
m, M = time_maxmin(c_a, m, M)
m, M = time_maxmin(c_q, m, M)
print(m, M)

338213.2991666667 404798.1744444444


In [18]:
A = nx.MultiDiGraph()
B = nx.MultiDiGraph()
C = nx.MultiDiGraph()
A=nx.from_pandas_dataframe(a_q, 'u', 'v', ['t', 'type'])
B=nx.from_pandas_dataframe(c_a, 'u', 'v', ['t', 'type'])
C=nx.from_pandas_dataframe(c_q, 'u', 'v', ['t', 'type'])
G = nx.compose(A,B)
G = nx.compose(G,C)

AttributeError: module networkx has no attribute from_pandas_dataframe

In [ ]:
#for function purposes
def create_graph(G, data, time_window, kind): #kind of link: a_q, c_a, c_q
    n_rows=len(data)
    for i in tqdm(range(n_rows)):
        u=c2_q.iloc[i][0]
        v=c2_q.iloc[i][1]
        time=c2_q.iloc[i][2]
        
        if time_window[0] <= time <= time_window[1] and (node_loops ):
            G.add_edge(u, v, time=t type=kind)
    

In [ ]:
G = nx.MultiDiGraph()
time_window = (M/2,M/2+1000)
kinds=['a_q', 'c_a', 'c_q']